In [18]:
import pandas as pd
import numpy as np
import datetime
import tqdm
import matplotlib.pyplot as plt
import pytictoc
import multiprocessing
from collections import defaultdict, OrderedDict
from tqdm import tqdm_notebook
import itertools

In [4]:
import os
import re
FIRST_N = 100
N_THREADS = 5
test_folder = '../../data/raw_splits/test/'
test_files = sorted([x for x in os.listdir(test_folder) if not '.pkl' in x], key = lambda x: int(re.sub('[^0-9]', '', x)))
test_files = [os.path.join(test_folder, x) for x in test_files]
print(f'Length of test files is {len(test_files)}')
train_folder = '../../data/raw_splits/train/'
train_files = sorted([x for x in os.listdir(train_folder) if not '.pkl' in x], key = lambda x: int(re.sub('[^0-9]', '', x)))
train_files = [os.path.join(train_folder, x) for x in train_files]
print(f'Length of train files is {len(train_files)}')

Length of test files is 26
Length of train files is 51


---

In [25]:
%%time
df = pd.read_feather(train_files[0])

CPU times: user 2.9 s, sys: 2.17 s, total: 5.07 s
Wall time: 5.06 s


In [26]:
df = df[['user_id', 'short_date', 'amount_original']].replace('null', np.nan).dropna()
df = df.rename(columns = {'amount_original': 'amount'}).sort_values(['user_id', 'short_date'])
df['amount'] = df['amount'].astype(float).astype(int)

In [69]:
#говнокод
dates_big_list = \
[int(f'201710%0.2d' % x) for x in list(range(28, 32))] + \
[int(f'201711%0.2d' % x) for x in list(range(1, 31))] + \
[int(f'201712%0.2d' % x) for x in list(range(1, 6))]

In [104]:
quants_range = np.arange(0.1, 1.1, 0.1)
STARTING_NDAY = 30 # с какого дня статуем цикл
NDAYS = 30 # за какой временной промежуток берем медиану
quants_names = ['amount_%s_days_quantile_%0.1f' % (NDAYS, q) for q in quants_range]
assert STARTING_NDAY >= NDAYS

---

In [96]:
grouper = itertools.groupby(zip(df['user_id'], df['short_date'], df['amount']), key = lambda x: x[0])
grouping_result = defaultdict(dict)
for uid, values in tqdm_notebook(grouper, total = df['user_id'].nunique()):
    (_, dates, amounts) = zip(*values)
    date_amount_dict = defaultdict(list)
    # сначала сделаем dict из даты в качестве ключа и amount каждой транзакции пользователя за этот день
    for date, amount_iter in itertools.groupby(zip(dates, amounts), key = lambda x: x[0]):
        _, day_amount_list = list(zip(*amount_iter))
        date_amount_dict[date] = day_amount_list
    # при правке интервалов даты поправить STARTING_NDAY, NDAYS
    for n in range(STARTING_NDAY, len(dates_big_list)):
        calculating_dates = dates_big_list[n - NDAYS: n]
        calculating_list = list(itertools.chain.from_iterable((date_amount_dict[x] for x in calculating_dates)))
        calc_quantile = lambda q: np.percentile(calculating_list, q * 100) if len(calculating_list) else 0
        quantiles = [calc_quantile(q) for q in quants_range]
        grouping_result[uid][dates_big_list[n]] = quantiles

HBox(children=(IntProgress(value=0, max=3867), HTML(value='')))

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



---

In [106]:
df_grouped_list = [[(uid, date, *quantiles) for date, quantiles in date_dict.items()] \
                   for uid, date_dict in grouping_result.items()]

In [108]:
df_grouped = pd.DataFrame(list(itertools.chain.from_iterable(df_grouped_list)), 
                          columns = ['user_id', 'short_date'] + quants_names)

In [117]:
df_grouped

,user_id,short_date,amount_30_days_quantile_0.1,amount_30_days_quantile_0.2,amount_30_days_quantile_0.3,amount_30_days_quantile_0.4,amount_30_days_quantile_0.5,amount_30_days_quantile_0.6,amount_30_days_quantile_0.7,amount_30_days_quantile_0.8,amount_30_days_quantile_0.9,amount_30_days_quantile_1.0
0,100197,20171127,975.0,1800.0,2700.0,3464.4,4837.0,5940.0,7500.0,14800.0,37000.0,100000.0
1,100197,20171128,975.0,1800.0,2700.0,3464.4,4837.0,5940.0,7500.0,14800.0,37000.0,100000.0
2,100197,20171129,1000.0,2000.0,3000.0,3774.0,5900.0,6000.0,11000.0,30000.0,50000.0,100000.0
3,100197,20171130,1000.0,2000.0,3000.0,3774.0,5900.0,6000.0,11000.0,30000.0,50000.0,100000.0
4,100197,20171201,750.0,1000.0,3000.0,3774.0,5900.0,6000.0,11000.0,30000.0,50000.0,100000.0
5,100197,20171202,725.0,950.0,2400.0,3464.4,4887.0,8000.0,16700.0,34000.0,55000.0,100000.0
6,100197,20171203,725.0,950.0,2400.0,3464.4,4887.0,8000.0,16700.0,34000.0,55000.0,100000.0
7,100197,20171204,725.0,950.0,2400.0,3464.4,4887.0,8000.0,16700.0,34000.0,55000.0,100000.0
8,100197,20171205,700.0,2100.0,3309.6,4219.2,6000.0,10000.0,22400.0,38000.0,60000.0,100000.0
9,10022990,20171127,148.8,192.4,621.2,2042.2,4636.0,5250.0,5314.0,5664.0,11800.0,15000.0
